In [ ]:
import wallaroo

In [ ]:
wl = wallaroo.Client()

In [ ]:
ws = wl.list_workspaces()
wl.list_workspaces()

# only run first time

In [ ]:
#wrk = wl.create_workspace('test')

In [ ]:
ws = wl.list_workspaces()
for w in ws:
    print(w.name())
    if w.name() == 'dish-pd-eks-ad':
        wl.set_current_workspace(w)

In [ ]:
wl.get_current_workspace()

# Upload and run our model

For this example we will be using an open source model that uses an Aloha CNN LSTM model for classifiying Domain names.

## Config 
before deploying an inference engine we will set the configuration of the engine.
To do this we will use the wallaroo DeploymentConfigBuilder() and fill in the options listed below to determine what the properties of our inference engine will be

note: this will not start the process of building anything in the kubernetes cluster yet. we are just setting the deployment configuration we will want to use later.
- replica_count - 1 => when deployed this will have a single inference engine
- cpus - 14 => each inference engine will have 14 cpus
- memory - 50Gi => each inference engine will have 50 Gb of memory

# Recommedations
for this we are going to create following deployment_configration: 
 - deployment_config
 -- this config will use a single replica with 4 cpus and 8 Gb of memory and will be used for our deployment later



In [ ]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(4).memory("4Gi").build()

# Upload model

In [ ]:
model = wl.upload_model('node-autoencoder-ad', "./node_autoencoder_ad_v0.0.2_training_2022_9_29.onnx")

# Deploy a model
Now that we have a model that we want to use we will create a deployment for it. 

We will tell the deployment we are using a tensorflow model and give the deployment name and the configuration we want for the deployment

# Example

In [ ]:
p = wl.build_pipeline('node-autoencoder-ad')
p = p.add_model_step(model)
pipeline = p.deploy(deployment_config = deployment_config)

# Successful deployment
now that we have a deployment running we start inferencing
(if you are still running out of resources check to make sure all other deployments have been taken down)

## infer 1 row
to test we will infer a single row of data and see the results using the infer_from_file tool
we will pass a single encoded url into the inference engine and print the results back out

In [ ]:
pipeline.infer_from_file("data.txt")

## Run larger batch
Now we will run a larger batch. We have prepared a 1,000 inference file and a 25,000 inference file
once the results are finished they will be placed in a file titled curl_response.txt 


In [ ]:
pipeline._deployment._url()

In [ ]:
!curl -X POST {pipeline._deployment._url()} -H "Content-Type:application/json" --data @test_data10.json

In [ ]:
model2 = wl.upload_model('ccfraud-xgb', './models/xgboost_ccfraud.onnx')
pipeline.replace_with_model_step(index=0, model=model2).deploy()

## Undeploy model
This will take down our inference engine. and free up the resources in kubernetes
- Note that if the pipeline variable is unchanged deployment.deploy() will restart the inference engine in the same configuration

In [ ]:
pipeline.undeploy()